In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install graphviz
!pip3 install dask
!pip install "dask[complete]"
!pip3 install toolz
!pip3 install cloudpickle
!pip3 install folium
!pip install gpxpy
!pip3 install pyroomacoustics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 13.7 MB/s 
  Created wheel for gpxpy: filename=gpxpy-1.5.0-py3-none-any.whl size=42899 sha256=aa8e7264735e7d65fb3c4fe610dc0c50afe4e32fc8877bb72fe3482afd82e7c8
  Stored in directory: /root/.cache/pip/wheels/78/49/b6/d1851fcb4ee10fa784325d7fcf6c97789157c706ab3a7

In [ ]:
import os
import time
import math
import scipy
import pickle
import warnings
import datetime
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import xgboost as xgb
import matplotlib.pylab as plt

# This library helps to open street map
import folium
import dask.dataframe as dd

# This library is used while we calculate the straight line distance between two (lat, lon) pairs
# in miles. Get the haversine distance
import gpxpy.geo

# Used to determine the size of plots
from matplotlib import rcParams

# https://pyroomacoustics.readthedocs.io/en/pypi-release/pyroomacoustics.doa.detect_peaks.html
# Used for detection of peaks
from pyroomacoustics.doa.detect_peaks import detect_peaks

# Download migwin: https://mingw-w64.org/doku.php/download/mingw-builds
# Install it in your system and keep the path, migw_path ='installed path'
# mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
# os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression

from prettytable import PrettyTable
from datetime import datetime as dt

matplotlib.use('nbagg');
warnings.filterwarnings("ignore");
%matplotlib inline

In [ ]:
path = "/content/drive/MyDrive/yellow_tripdata_2016-01.csv"
df = pd.read_csv(path)

In [ ]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8


In [ ]:
sampled_df=df.sample(n = 10000, replace = True, random_state=5)

In [ ]:
sampled_df.shape

(10000, 19)

In [ ]:
from google.colab import files
sampled_df.to_csv('sdf.csv')
files.download('sdf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
path_sdf="/content/sdf.csv"
month = dd.read_csv(path_sdf, dtype={'tip_amount': 'float64',
       'tolls_amount': 'float64'})
print(month.columns)

month.head()

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'RatecodeID',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount'],
      dtype='object')


,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2115790,1,2016-01-10 20:41:48,2016-01-10 21:16:32,1,7.20,-73.950821,40.779346,1,N,-73.952148,40.776451,1,27.5,0.5,0.5,5.75,0.0,0.3,34.55
1,1956982,1,2016-01-09 22:40:59,2016-01-09 22:55:51,1,2.30,-73.986397,40.722317,1,N,-73.984375,40.747650,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35
2,2758032,1,2016-01-12 13:07:24,2016-01-12 13:20:40,1,2.40,-73.979179,40.764565,1,N,-74.004593,40.741966,1,11.0,0.0,0.5,1.50,0.0,0.3,13.30
3,558088,2,2016-01-02 17:21:08,2016-01-02 17:27:56,2,1.09,-73.982010,40.728081,1,N,-73.995003,40.736462,2,6.5,0.0,0.5,0.00,0.0,0.3,7.30
4,10174183,2,2016-01-07 14:47:12,2016-01-07 15:03:18,1,1.32,-73.970100,40.764744,1,N,-73.957932,40.779133,1,10.5,0.0,0.5,2.26,0.0,0.3,13.56


In [ ]:
outlier_locations = month[((month.pickup_longitude <= -74.15) | (month.pickup_latitude <= 40.5774)| \
    (month.pickup_longitude >= -73.7004) | (month.pickup_latitude >= 40.9176))]
print("Number of Outlier Locations:", len(outlier_locations))
#map for new york bounded by -74.15 and 40.5774

Number of Outlier Locations: 161


In [ ]:
map_osm = folium.Map(location=[40.734695, -73.990372], tiles='Stamen Toner')

In [ ]:
sample_locations = outlier_locations
for i,j in sample_locations.iterrows():
    if int(j['pickup_latitude']) != 0:
        folium.Marker(list((j['pickup_latitude'],j['pickup_longitude']))).add_to(map_osm)

map_osm

#mapping pickup outliers

In [ ]:
outlier_locations = month[((month.dropoff_longitude <= -74.15) | (month.dropoff_latitude <= 40.5774)| \
    (month.dropoff_longitude >= -73.7004) | (month.dropoff_latitude >= 40.9176))]
print("Number of Outlier Locations:", len(outlier_locations))

Number of Outlier Locations: 159


In [ ]:
map_osm = folium.Map(location=[40.734695, -73.990372], tiles='Stamen Toner')

In [ ]:
sample_locations = outlier_locations
for i,j in sample_locations.iterrows():
    if int(j['pickup_latitude']) != 0:
        folium.Marker(list((j['dropoff_latitude'],j['dropoff_longitude']))).add_to(map_osm)

map_osm

#mapping dropff outlier locations

In [ ]:
def convert_to_unix(s):
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())
def return_with_trip_times(month):
    duration = month[['tpep_pickup_datetime','tpep_dropoff_datetime']].compute()

    duration_pickup = [convert_to_unix(x) for x in duration['tpep_pickup_datetime'].values]
    duration_drop = [convert_to_unix(x) for x in duration['tpep_dropoff_datetime'].values]

    durations = (np.array(duration_drop) - np.array(duration_pickup))/float(60)
    new_frame = month[['passenger_count','trip_distance','pickup_longitude','pickup_latitude',
        'dropoff_longitude','dropoff_latitude','total_amount']].compute()

    new_frame['trip_times'] = durations
    new_frame['pickup_times'] = duration_pickup
    new_frame['dropoff_times']= duration_drop
    new_frame['Speed'] = 60*(new_frame['trip_distance']/new_frame['trip_times'])

    return new_frame

frame_with_durations = return_with_trip_times(month)
frame_with_durations.head()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,trip_times,pickup_times,dropoff_times,Speed
0,1,7.20,-73.950821,40.779346,-73.952148,40.776451,34.55,34.733333,1.452459e+09,1.452461e+09,12.437620
1,1,2.30,-73.986397,40.722317,-73.984375,40.747650,15.35,14.866667,1.452379e+09,1.452380e+09,9.282511
2,1,2.40,-73.979179,40.764565,-74.004593,40.741966,13.30,13.266667,1.452604e+09,1.452605e+09,10.854271
3,2,1.09,-73.982010,40.728081,-73.995003,40.736462,7.30,6.800000,1.451755e+09,1.451756e+09,9.617647
4,1,1.32,-73.970100,40.764744,-73.957932,40.779133,13.56,16.100000,1.452178e+09,1.452179e+09,4.919255


In [ ]:
def remove_outliers(new_frame):
    a = new_frame.shape[0]
    print("Number of pickup records:", a)

    temp_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
        (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
        ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
        (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]
    b = temp_frame.shape[0]
    print("Number of outlier coordinates lying outside NY boundaries:", (a-b))

    temp_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    c = temp_frame.shape[0]
    print("Number of outliers from trip times analysis:", (a-c))

    temp_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    d = temp_frame.shape[0]
    print ("Number of outliers from trip distance analysis:", (a-d))

    temp_frame = new_frame[(new_frame.Speed <= 45.31) & (new_frame.Speed >= 0)]
    e = temp_frame.shape[0]
    print ("Number of outliers from speed analysis:", (a-e))

    temp_frame = new_frame[(new_frame.total_amount < 1000) & (new_frame.total_amount > 0)]
    f = temp_frame.shape[0]
    print ("Number of outliers from fare analysis:", (a-f))


    new_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
        (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
        ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
        (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]
    new_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    new_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    new_frame = new_frame[(new_frame.Speed < 45.31) & (new_frame.Speed > 0)]
    new_frame = new_frame[(new_frame.total_amount < 1000) & (new_frame.total_amount > 0)]

    print ("Total outliers removed", a-new_frame.shape[0])
    return new_frame

    print (new_frame.shape())

In [ ]:
print ("Removing outliers in the month of Jan-2016")
frame_with_durations_outliers_removed = remove_outliers(frame_with_durations)
print("Fraction of data points that remain after removing outliers",
    float(len(frame_with_durations_outliers_removed)) / len(frame_with_durations))

Removing outliers in the month of Jan-2016
Number of pickup records: 10000
Number of outlier coordinates lying outside NY boundaries: 189
Number of outliers from trip times analysis: 14
Number of outliers from trip distance analysis: 75
Number of outliers from speed analysis: 21
Number of outliers from fare analysis: 8
Total outliers removed 278
Fraction of data points that remain after removing outliers 0.9722


In [ ]:
import numpy as np
#from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
frame_with_durations_outliers_removed.head()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,trip_times,pickup_times,dropoff_times,Speed
0,1,7.20,-73.950821,40.779346,-73.952148,40.776451,34.55,34.733333,1.452459e+09,1.452461e+09,12.437620
1,1,2.30,-73.986397,40.722317,-73.984375,40.747650,15.35,14.866667,1.452379e+09,1.452380e+09,9.282511
2,1,2.40,-73.979179,40.764565,-74.004593,40.741966,13.30,13.266667,1.452604e+09,1.452605e+09,10.854271
3,2,1.09,-73.982010,40.728081,-73.995003,40.736462,7.30,6.800000,1.451755e+09,1.451756e+09,9.617647
4,1,1.32,-73.970100,40.764744,-73.957932,40.779133,13.56,16.100000,1.452178e+09,1.452179e+09,4.919255


In [ ]:
pickup_df=frame_with_durations_outliers_removed[['pickup_times','pickup_longitude','pickup_latitude']].copy()

In [ ]:
pickup_df.head()

,pickup_times,pickup_longitude,pickup_latitude
0,1.452459e+09,-73.950821,40.779346
1,1.452379e+09,-73.986397,40.722317
2,1.452604e+09,-73.979179,40.764565
3,1.451755e+09,-73.982010,40.728081
4,1.452178e+09,-73.970100,40.764744


In [ ]:
from datetime import timedelta
! pip install pyproj --no-binary pyproj


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 213 kB 15.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 212 kB 53.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 182 kB 40.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 168 kB 50.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 663 kB 61.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 663 kB 29.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |█████████████

In [ ]:
from pyproj import Proj

In [ ]:
! pip install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=5846c7f3cd20eed6670e797ce52fa8b9f7487ce08154b066b863dfa5e991c954
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [ ]:
import utm

In [ ]:
utm_pickup=pickup_df[['pickup_times']].copy()

In [ ]:
lon=pickup_df['pickup_longitude'].values
lat=pickup_df['pickup_latitude'].values

In [ ]:
mat = utm.from_latlon(lat, lon)
mat[0]
mat[1]

array([4514792.35836952, 4508426.34406576, 4513123.21085755, ...,
       4514437.24092017, 4512716.57913617, 4510227.73016971])

In [ ]:
utm_pickup['utm_longitude']=mat[1].tolist()
utm_pickup['utm_latitude']=mat[0].tolist()

In [ ]:
utm_pickup.head()

,pickup_times,utm_longitude,utm_latitude
0,1.452459e+09,4.514792e+06,588531.912117
1,1.452379e+09,4.508426e+06,585603.013695
2,1.452604e+09,4.513123e+06,586158.022439
3,1.451755e+09,4.509070e+06,585966.093884
4,1.452178e+09,4.513152e+06,586924.073745


In [ ]:
frame_with_durations_outliers_removed.head()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,trip_times,pickup_times,dropoff_times,Speed
0,1,7.20,-73.950821,40.779346,-73.952148,40.776451,34.55,34.733333,1.452459e+09,1.452461e+09,12.437620
1,1,2.30,-73.986397,40.722317,-73.984375,40.747650,15.35,14.866667,1.452379e+09,1.452380e+09,9.282511
2,1,2.40,-73.979179,40.764565,-74.004593,40.741966,13.30,13.266667,1.452604e+09,1.452605e+09,10.854271
3,2,1.09,-73.982010,40.728081,-73.995003,40.736462,7.30,6.800000,1.451755e+09,1.451756e+09,9.617647
4,1,1.32,-73.970100,40.764744,-73.957932,40.779133,13.56,16.100000,1.452178e+09,1.452179e+09,4.919255


In [ ]:
dropoff_df=frame_with_durations_outliers_removed[['dropoff_times','dropoff_longitude','dropoff_latitude']].copy()

In [ ]:
utm_dropoff=dropoff_df[['dropoff_times']].copy()

In [ ]:
lon_d=dropoff_df['dropoff_longitude'].values
lat_d=dropoff_df['dropoff_latitude'].values

In [ ]:
mat1 = utm.from_latlon(lat_d, lon_d)
mat1[0]
mat1[1]

array([4514469.61179506, 4511240.50200911, 4510589.99469279, ...,
       4511636.48557974, 4514654.28712508, 4510626.38111539])

In [ ]:
utm_dropoff['utm_longitude']=mat1[1].tolist()
utm_dropoff['utm_latitude']=mat1[0].tolist()

In [ ]:
utm_dropoff.head()

,dropoff_times,utm_longitude,utm_latitude
0,1.452461e+09,4.514470e+06,588423.731362
1,1.452380e+09,4.511241e+06,585741.232124
2,1.452605e+09,4.510590e+06,584041.523453
3,1.451756e+09,4.509988e+06,584858.220972
4,1.452179e+09,4.514761e+06,587932.177111
